# Video Processing for BT Lab
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import re

In [2]:
# local variables

p3225_file_path = {
    "video folder" : "DI_CAMERA_P3225",
    "1st level" : ["Final"],
    "2nd level" : "Patient Name",
    "3rd level" : [2, 3],
    "4th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"}, 
    "5th level" : {"H" : "Hold Breath",
                    "R": "Relaxed"} 
}

fur_file_path = {
    "video folder" : "DI_THERMAL_FLIR",
    "1st level" : [""],
    "2nd level" : "Patient Number",
    "3rd level" : [2, 3],
    "4th level" : { "H": "Hold Breath",
                        "R": "Relaxed"}, 
    "5th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"} 
}

root_path = "E:/Christian/DI_centre_structured"
repo_dir = os.getcwd()

In [3]:
def get_video_filenames(patient_path:str, level=0) -> None:
    file_paths = []

    os.chdir(patient_path)
    if level < 4:
        for entry in os.scandir(patient_path):
            if entry.is_file():
                file_paths.append(entry.path)
            elif entry.is_dir():
                file_paths.extend(get_video_filenames(entry.path, level+1))

    return file_paths

In [4]:
def get_patient_id(curr_dir:str) -> None:
    os.chdir(curr_dir)
    # patient_names = [folder_content for folder_content in os.listdir() if not folder_content.endswith(".mp4")]
    patient_names = [folder_content for folder_content in os.listdir() if os.path.isdir(folder_content)]
    return patient_names

In [5]:
def filter_filenames(video_filenames:list[str], keyword:str) -> list[str]:
    filtered_filenames = []
    for filename in video_filenames:
        if not re.search(r"\b{}\b".format(keyword), filename, flags=re.IGNORECASE):
            filtered_filenames.append(filename)
    return filtered_filenames

In [6]:
def save_filenames(root_dir:str, save_dir:str, video_name:str, keywords:list[str], patient_list:list[str]) -> None:
    log_file = f"{save_dir}/{video_name}.log"
    log_file = log_file.replace("\\", "/")

    if os.path.exists(log_file):
        os.remove(log_file)

    for patient_count, patient_name in enumerate(patient_list):
        patient_folder = root_dir + "/" + patient_name
        all_patient_mp4_files = get_video_filenames(patient_folder)
        filtered_filenames = all_patient_mp4_files

        for word in keywords:
            filtered_filenames = filter_filenames(filtered_filenames, word)

        with open(log_file, "a") as mp4_data:
            for mp4_file in filtered_filenames:
                mp4_data.write(f"{mp4_file}\n")

        # if patient_count == 1:
        #     break

## Retrieve all filenames (stage 1.1)

Filter out unwanted filenames and save them in log files

In [7]:
""" 
    get filenames for rgb camera
"""

video_folder = p3225_file_path["video folder"]
all_videos_folder = root_path + "/" + video_folder + "/" + p3225_file_path["1st level"][0]
patient_names = get_patient_id(all_videos_folder)
patient_names

['Arun',
 'Daniel',
 'Eddy',
 'Farooq',
 'Frank',
 'Frank2',
 'Illia',
 'Illia - 2',
 'Jay',
 'John Brunton',
 'John McNeil',
 'Joseph',
 'Karlii',
 'Kate',
 'Krystyn',
 'Melissa',
 'Nick',
 'Rachael B',
 'Riyan',
 'Sara']

In [8]:
records_folder = repo_dir + "/record"
keywords = ["media_media.amp-.mp4"]
save_filenames(all_videos_folder, records_folder, video_folder, keywords, patient_names)

In [11]:
""" 
    get filenames for thermal camera
"""

video_folder = fur_file_path["video folder"]
all_videos_folder = root_path + "/" + video_folder + "/" + fur_file_path["1st level"][0]
patient_names = get_patient_id(all_videos_folder)
patient_names

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [12]:
records_folder = repo_dir + "/record"
keywords = [".Bmp", ".png"]
save_filenames(all_videos_folder, records_folder, video_folder, keywords, patient_names)

## Extract information from the exported log files and create containers for the metadata (stage 1.2)

In [39]:
def extract_info_from_filepath(filepaths:list[str]) -> list[dict]:
    all_videos = []
    file_info_from_path = {
        0 : "patient id",
        1 : "distance",
        2 : "blanket",
        3 : "breathing",
        4 : "filename"
    }
    metadata_info = {
        5 : "frames",
        6 : "video length",
        7 : "old fps"
    }

    splice_stop = 5

    for filepath in filepaths:
        filepath_no_backslash = filepath.replace("\\", "/")
        filepath_to_list = filepath_no_backslash.split("/")
        filepath_to_list = filepath_to_list[-splice_stop:]
        patient_info = {tag:filepath_to_list[key] for key, tag in file_info_from_path.items() if key<splice_stop}
        patient_info = {**patient_info, **metadata_info}
        all_videos.append(patient_info)

    return all_videos

In [21]:
"""
    Read log files
"""

os.chdir(records_folder)
log_files = [dir_file for dir_file in os.listdir() if dir_file.endswith(".log")]
video_files = []

for log_file in log_files:
    try:
        with open(log_file, "r") as log_file_data:
            log_read = log_file_data.read().split("\n")
            video_files.append(log_read)
    except FileNotFoundError as e:
        print(type(e))

In [40]:
"""
    make dictionary out of filenames
"""
test_list = [video_files[0][0]]
metadata_rgb = extract_info_from_filepath(test_list)
metadata_rgb

[{'patient id': 'Arun',
  'distance': '2 Meters',
  'blanket': 'With Blankets',
  'breathing': 'Hold Breath',
  'filename': 'Arun2.mp4',
  5: 'frames',
  6: 'video length',
  7: 'old fps'}]